# Learning: Advanced Planning Algorithms

This notebook guides you through implementing advanced path planning algorithms like A* and RRT. **You'll need to fill in the missing code!**

## What You'll Learn

1. Understanding graph-based planning (A*)
2. Understanding sampling-based planning (RRT)
3. Implementing A* planner
4. Implementing RRT planner
5. Comparing different planning approaches

## Instructions

- Read each section carefully
- Look for `# TODO:` comments - these indicate where you need to write code
- Fill in the `...` placeholders
- Run cells as you complete them to test your work

## Prerequisites

- Understanding of planners (see [Planner Tutorial](../tutorials/planner_tutorial.ipynb))
- Understanding of costmaps (see [Costmap Tutorial](../tutorials/costmap_tutorial.ipynb))
- Graph algorithms knowledge (helpful but not required)
- Basic understanding of A* and RRT algorithms

In [ ]:
import sys

sys.path.insert(0, '../../src')

import matplotlib.pyplot as plt
import numpy as np

from simple_autonomous_car import (
    BasePlanner,
    Car,
    CarState,
    GridCostmap,
    Track,
)

## Step 1: A* Planner

A* is an optimal graph search algorithm that finds the shortest path using:
- **g(n)**: Cost from start to node n
- **h(n)**: Heuristic estimate from node n to goal
- **f(n) = g(n) + h(n)**: Total estimated cost

**TODO**: Implement the A* algorithm below!

In [ ]:
import heapq


class AStarPlanner(BasePlanner):
    """
    A* path planning algorithm.

    TODO: Fill in the implementation!
    """

    def __init__(self, costmap, resolution: float = 1.0, name: str = "astar_planner"):
        """
        Initialize A* planner.

        TODO: Store costmap and resolution
        """
        super().__init__(name=name)
        self.costmap = costmap
        self.resolution = resolution

    def _heuristic(self, pos1: np.ndarray, pos2: np.ndarray) -> float:
        """
        Heuristic function (Euclidean distance).

        TODO: Return Euclidean distance between pos1 and pos2
        """
        return ...

    def _get_neighbors(self, pos: np.ndarray, car_state: CarState) -> list:
        """
        Get neighboring grid cells.

        TODO:
        1. Convert position to grid coordinates (use costmap._world_to_grid if available, or implement)
        2. Generate 8 neighbors (up, down, left, right, and diagonals)
        3. Convert neighbors back to world coordinates
        4. Filter out neighbors with high cost (> 0.5)
        5. Return list of valid neighbor positions

        HINT: For simplicity, you can use a fixed step size (e.g., resolution)
        """
        neighbors = []
        step = self.resolution

        # TODO: Generate 8-directional neighbors
        for dx in [-step, 0, step]:
            for dy in [-step, 0, step]:
                if dx == 0 and dy == 0:
                    continue

                neighbor_pos = pos + np.array([dx, dy])

                # TODO: Check cost at neighbor position
                cost = ...
                if cost <= 0.5:  # Free or low cost
                    neighbors.append(neighbor_pos)

        return neighbors

    def plan(
        self,
        car_state: CarState,
        perception_data: dict = None,
        costmap = None,
        goal = None,
    ) -> np.ndarray:
        """
        Plan path using A* algorithm.

        TODO: Implement A* search:
        1. If no costmap or goal, return empty array
        2. Initialize:
           - start = car_state.position()
           - open_set = priority queue with (f_score, start)
           - came_from = {} (to reconstruct path)
           - g_score = {start: 0}
           - f_score = {start: heuristic(start, goal)}
        3. While open_set not empty:
           - Pop node with lowest f_score
           - If node == goal, reconstruct path and return
           - For each neighbor:
             - tentative_g = g_score[node] + cost_to_neighbor
             - If tentative_g < g_score[neighbor] (or neighbor not visited):
               - came_from[neighbor] = node
               - g_score[neighbor] = tentative_g
               - f_score[neighbor] = g_score[neighbor] + heuristic(neighbor, goal)
               - Add neighbor to open_set
        4. If no path found, return empty array

        HINT: Use heapq for priority queue, use tuple keys for positions
        """
        if costmap is None:
            costmap = self.costmap

        if costmap is None or not costmap.enabled or goal is None:
            return np.array([]).reshape(0, 2)

        # TODO: Initialize A* search
        start = ...

        # Priority queue: (f_score, position)
        open_set = [(self._heuristic(start, goal), tuple(start))]
        heapq.heapify(open_set)

        came_from = {}
        g_score = {tuple(start): 0.0}
        f_score = {tuple(start): self._heuristic(start, goal)}

        visited = set()

        # TODO: A* search loop
        while open_set:
            # Pop node with lowest f_score
            current_f, current = heapq.heappop(open_set)
            current = np.array(current)

            # TODO: Check if goal reached
            if np.linalg.norm(current - goal) < ...:
                # Reconstruct path
                path = []
                node = tuple(goal)
                while node in came_from:
                    path.append(list(node))
                    node = came_from[node]
                path.append(list(start))
                path.reverse()
                return np.array(path)

            visited.add(tuple(current))

            # TODO: Explore neighbors
            neighbors = self._get_neighbors(current, car_state)
            for neighbor in neighbors:
                neighbor_key = tuple(neighbor)

                if neighbor_key in visited:
                    continue

                # TODO: Calculate tentative g_score
                tentative_g = ...

                # TODO: Update if better path found
                if neighbor_key not in g_score or tentative_g < g_score[neighbor_key]:
                    came_from[neighbor_key] = ...
                    g_score[neighbor_key] = ...
                    f_score[neighbor_key] = ...

                    # Add to open set if not already there
                    heapq.heappush(open_set, (f_score[neighbor_key], neighbor_key))

        # No path found
        return np.array([]).reshape(0, 2)

print("✓ A* planner structure created!")
print("  Now fill in the TODOs!")

## Step 2: RRT Planner

RRT (Rapidly-exploring Random Tree) is a sampling-based planner that builds a tree by randomly sampling the configuration space.

**TODO**: Implement the RRT algorithm below!

In [ ]:
class RRTPlanner(BasePlanner):
    """
    RRT (Rapidly-exploring Random Tree) planner.

    TODO: Fill in the implementation!
    """

    def __init__(self, costmap, max_iterations: int = 500, step_size: float = 2.0, name: str = "rrt_planner"):
        """
        Initialize RRT planner.

        TODO: Store parameters
        """
        super().__init__(name=name)
        self.costmap = costmap
        self.max_iterations = max_iterations
        self.step_size = step_size

    def _random_sample(self, car_state: CarState) -> np.ndarray:
        """
        Generate a random sample in the costmap bounds.

        TODO:
        1. Get costmap bounds (depends on frame)
        2. Generate random point within bounds
        3. If ego frame, transform to global
        4. Return random point
        """
        if self.costmap.frame == "ego":
            # Ego frame: sample in ego bounds, then transform
            x = np.random.uniform(-self.costmap.width/2, self.costmap.width/2)
            y = np.random.uniform(-self.costmap.height/2, self.costmap.height/2)
            np.array([x, y])
            return ...
        else:
            # Global frame
            origin = self.costmap.origin
            x = np.random.uniform(origin[0] - self.costmap.width/2, origin[0] + self.costmap.width/2)
            y = np.random.uniform(origin[1] - self.costmap.height/2, origin[1] + self.costmap.height/2)
            return np.array([x, y])

    def _nearest_node(self, tree: list, point: np.ndarray) -> int:
        """
        Find nearest node in tree to given point.

        TODO:
        1. Calculate distances from point to all nodes in tree
        2. Return index of nearest node
        """
        if len(tree) == 0:
            return -1

        # TODO: Find nearest node
        return ...

    def _steer(self, from_pos: np.ndarray, to_pos: np.ndarray) -> np.ndarray:
        """
        Steer from one position towards another (limited by step_size).

        TODO:
        1. Calculate vector from from_pos to to_pos
        2. If distance > step_size, limit to step_size
        3. Return new position
        """
        distance = ...

        if distance <= self.step_size:
            return to_pos
        else:
            # TODO: Limit to step_size
            return from_pos + ...

    def _is_collision_free(self, pos: np.ndarray, car_state: CarState) -> bool:
        """
        Check if position is collision-free.

        TODO: Get cost at position and check if < 0.5
        """
        cost = ...
        return cost < 0.5

    def plan(
        self,
        car_state: CarState,
        perception_data: dict = None,
        costmap = None,
        goal = None,
    ) -> np.ndarray:
        """
        Plan path using RRT algorithm.

        TODO: Implement RRT:
        1. If no costmap or goal, return empty array
        2. Initialize tree with start node: [car_state.position()]
        3. For max_iterations:
           - Sample random point
           - Find nearest node in tree
           - Steer from nearest towards random (new_node)
           - If new_node is collision-free:
             - Add to tree
             - If close to goal, connect to goal and return path
        4. If goal reached, reconstruct path from tree
        5. Return path or empty array

        HINT: Store tree as list of positions, and parent indices separately
        """
        if costmap is None:
            costmap = self.costmap

        if costmap is None or not costmap.enabled or goal is None:
            return np.array([]).reshape(0, 2)

        # TODO: Initialize RRT tree
        start = ...
        tree = [start]
        parents = [-1]  # -1 means root

        # TODO: RRT main loop
        for iteration in range(self.max_iterations):
            # Sample random point

            # Find nearest node
            nearest_idx = ...
            tree[nearest_idx]

            # Steer towards random point
            new_node = ...

            # TODO: Check collision
            if ...:
                # Add to tree
                tree.append(new_node)
                parents.append(nearest_idx)

                # TODO: Check if close to goal
                if np.linalg.norm(new_node - goal) < ...:
                    # Reconstruct path
                    path = []
                    idx = len(tree) - 1
                    while idx >= 0:
                        path.append(tree[idx])
                        idx = parents[idx]
                    path.reverse()
                    path.append(goal)
                    return np.array(path)

        # No path found
        return np.array([]).reshape(0, 2)

print("✓ RRT planner structure created!")
print("  Now fill in the TODOs!")

## Step 3: Test Your Planners

In [ ]:
# Setup test environment
track = Track.create_simple_track(length=80.0, width=40.0, track_width=5.0)
ground_truth_map = GroundTruthMap(track)
perceived_map = PerceivedMap(ground_truth_map)

from simple_autonomous_car import LiDARSensor

start_point, start_heading = track.get_point_at_distance(0.0)
car = Car(initial_state=CarState(x=start_point[0], y=start_point[1], heading=start_heading, velocity=8.0))

lidar = LiDARSensor(ground_truth_map, perceived_map, max_range=30.0, name="lidar")
car.add_sensor(lidar)

# Create costmap
costmap = GridCostmap(width=60.0, height=60.0, resolution=2.0, frame="ego")

# Update costmap
perception_data = car.sense_all(environment_data={"ground_truth_map": ground_truth_map})
costmap.update(perception_data, car.state)

# Define goal
goal = car.state.position() + np.array([20.0, 10.0])

print("✓ Test setup complete!")
print(f"  Start: ({car.state.x:.2f}, {car.state.y:.2f})")
print(f"  Goal: ({goal[0]:.2f}, {goal[1]:.2f})")

In [ ]:
# Test A* planner
astar_planner = AStarPlanner(costmap, resolution=2.0)

plan_astar = astar_planner.plan(car.state, costmap=costmap, goal=goal)

print(f"✓ A* plan: {len(plan_astar)} waypoints")
if len(plan_astar) > 0:
    print(f"  Path length: {np.sum(np.linalg.norm(np.diff(plan_astar, axis=0), axis=1)):.2f}m")

In [ ]:
# Test RRT planner
rrt_planner = RRTPlanner(costmap, max_iterations=300, step_size=2.0)

plan_rrt = rrt_planner.plan(car.state, costmap=costmap, goal=goal)

print(f"✓ RRT plan: {len(plan_rrt)} waypoints")
if len(plan_rrt) > 0:
    print(f"  Path length: {np.sum(np.linalg.norm(np.diff(plan_rrt, axis=0), axis=1)):.2f}m")

In [ ]:
# Visualize both planners
from simple_autonomous_car.visualization import plot_car

fig, axes = plt.subplots(1, 2, figsize=(16, 8))

# A* planner
ax = axes[0]
track.visualize(ax=ax, frame="global")
costmap.visualize(ax=ax, car_state=car.state, frame="global", alpha=0.3)
if len(plan_astar) > 0:
    astar_planner.visualize(ax=ax, car_state=car.state, plan=plan_astar, frame="global", color="green", label="A* Plan", show_waypoints=True)
ax.plot(goal[0], goal[1], "r*", markersize=15, label="Goal")
plot_car(car, ax=ax, show_heading=True)
ax.set_title("A* Planner")
ax.legend()

# RRT planner
ax = axes[1]
track.visualize(ax=ax, frame="global")
costmap.visualize(ax=ax, car_state=car.state, frame="global", alpha=0.3)
if len(plan_rrt) > 0:
    rrt_planner.visualize(ax=ax, car_state=car.state, plan=plan_rrt, frame="global", color="blue", label="RRT Plan", show_waypoints=True)
ax.plot(goal[0], goal[1], "r*", markersize=15, label="Goal")
plot_car(car, ax=ax, show_heading=True)
ax.set_title("RRT Planner")
ax.legend()

plt.tight_layout()
plt.show()

print("✓ Visualization complete!")

## Summary

Congratulations! You've implemented advanced planning algorithms! 🎉

### What You Learned

1. ✅ **A* algorithm**: Optimal graph search with heuristics
2. ✅ **RRT algorithm**: Sampling-based planning for complex environments
3. ✅ **Path reconstruction**: Building paths from search trees
4. ✅ **Collision checking**: Using costmaps for obstacle avoidance

### Key Concepts

- **A***: Optimal but can be slow for large state spaces
- **RRT**: Fast, good for complex obstacles, but not optimal
- **Heuristics**: Estimates that guide search (A*)
- **Sampling**: Random exploration of space (RRT)

### Next Steps

- Improve A* with better heuristics
- Implement RRT* (optimal variant of RRT)
- Add path smoothing
- Implement hybrid planners (A* + RRT)
- Add dynamic replanning